In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np

import sklearn
sklearn.show_versions()


System:
    python: 3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]
executable: C:\Users\Alicia BASSIERE\miniconda3\python.exe
   machine: Windows-10-10.0.19045-SP0

Python dependencies:
      sklearn: 1.3.0
          pip: 23.3
   setuptools: 68.0.0
        numpy: 1.26.0
        scipy: 1.11.3
       Cython: None
       pandas: 1.3.4
   matplotlib: 3.5.1
       joblib: 1.2.0
threadpoolctl: 2.2.0

Built with OpenMP: True


AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
folder_path = r"C:\Users\Alicia BASSIERE\OneDrive - GENES\Documents\Paper 02 - Mean Field\Data"
data = pd.read_csv(folder_path + r"\\auction_sample.csv" , index_col=0)
data = data[data['Sale/Purchase'] == "Sell"]
data

In [ ]:
# Group by 'Price' and calculate the cumulative sum of 'Volume'
cumulative_volume = data.groupby('Price')['Volume'].cumsum()

# Create a new DataFrame with 'Price' and 'Cumulative_Volume' columns
cumulative_data = pd.DataFrame({'Price': data['Price'], 'Cumulative_Volume': cumulative_volume})

# Drop duplicate rows to keep only unique 'Price' values
cumulative_data = cumulative_data.drop_duplicates(subset='Price', keep='last')

# Reset the index for the new DataFrame
cumulative_data.reset_index(drop=True, inplace=True)

# Sort the DataFrame by ascending 'Price'
cumulative_data = cumulative_data.sort_values(by='Price')

# Calculate the cumulative sum of 'Cumulative_Volume' and add it as a new column
cumulative_data['Cumulative_Sum'] = cumulative_data['Cumulative_Volume'].cumsum()

# Reset the index for the updated DataFrame
cumulative_data.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
print(cumulative_data)

In [ ]:
# Filter the initial DataFrame to keep rows where 'Hour' is 1
filtered_data1 = data[data['Hour'] == 1]

# Group by 'Price' and calculate the cumulative sum of 'Volume'
cumulative_volume = filtered_data1.groupby('Price')['Volume'].cumsum()/1000

# Create a new DataFrame with 'Price' and 'Cumulative_Volume' columns
cumulative_data = pd.DataFrame({'Price': filtered_data1['Price'], 'Cumulative_Volume': cumulative_volume})

# Drop duplicate rows to keep only unique 'Price' values
cumulative_data = cumulative_data.drop_duplicates(subset='Price', keep='last')

# Reset the index for the new DataFrame
cumulative_data.reset_index(drop=True, inplace=True)

# Sort the DataFrame by ascending 'Price'
cumulative_data = cumulative_data.sort_values(by='Price')

# Calculate the cumulative sum of 'Cumulative_Volume' and add it as a new column
cumulative_data['Cumulative_Sum'] = cumulative_data['Cumulative_Volume'].cumsum()

# Reset the index for the updated DataFrame
cumulative_data.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
print(cumulative_data)

# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(cumulative_data['Cumulative_Sum'], cumulative_data['Price'], marker='.', color='blue')
plt.title('Hourly Supply bid curve, 1AM')
plt.xlabel('Offered quantity of electricity')
plt.ylabel('Price €')
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Filter the initial DataFrame to keep rows where 'Hour' is 1
filtered_data14 = data[data['Hour'] == 14]

# Group by 'Price' and calculate the cumulative sum of 'Volume'
cumulative_volume = filtered_data14.groupby('Price')['Volume'].cumsum()

# Create a new DataFrame with 'Price' and 'Cumulative_Volume' columns
cumulative_data = pd.DataFrame({'Price': filtered_data14['Price'], 'Cumulative_Volume': cumulative_volume})

# Drop duplicate rows to keep only unique 'Price' values
cumulative_data = cumulative_data.drop_duplicates(subset='Price', keep='last')

# Reset the index for the new DataFrame
cumulative_data.reset_index(drop=True, inplace=True)

# Sort the DataFrame by ascending 'Price'
cumulative_data = cumulative_data.sort_values(by='Price')

# Calculate the cumulative sum of 'Cumulative_Volume' and add it as a new column
cumulative_data['Cumulative_Sum'] = cumulative_data['Cumulative_Volume'].cumsum()

# Reset the index for the updated DataFrame
cumulative_data.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
print(cumulative_data)

# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(cumulative_data['Cumulative_Sum'], cumulative_data['Price'], marker='.', color='green')
plt.title('Hourly Supply bid curve, 2PM (same day)')
plt.xlabel('Offered quantity of electricity')
plt.ylabel('Price €')
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Select the features for clustering
X = np.array(cumulative_data[['Price', 'Cumulative_Sum']].values)
X = np.array([[1, 2], [1, 4], [1, 0],[10, 2], [10, 4], [10, 0]])
print(X)
# Initialize and fit the k-means model
# n_clusters is set to 4 as you wanted to classify the dataset into 4 categories
kmeans = KMeans(n_clusters=4, random_state=0, n_init=10).fit(X)

# Predict the clusters
clusters = kmeans.predict(X)

# Assign the cluster back to your dataframe
cumulative_data['cluster'] = clusters

# Optionally, you can visualize the clustering (if it makes sense for your dataset)
plt.scatter(cumulative_data['price'], cumulative_data['electricity_offer'], c=cumulative_data['cluster'], cmap='viridis')
plt.xlabel('Price')
plt.ylabel('Electricity Offer')
plt.title('K-Means Clustering of the Dataset')
plt.show()